# TP2: Les modèles HMC pour la segmentation de texte
*MA202 Modèles de Markov*

*27 avril 2020*

Corentin Soubeiran

Chargé de TD: E. Azeraf 
Professeur: W. Pieczynski

<img src=https://upload.wikimedia.org/wikipedia/commons/4/45/Logo_ENSTA_Paris.jpg width="10%">


## Question 1:
--------------

Démontrons l'écriture du Forward et backward: 

### Forward:
------------


En adoptant les notations du TD nous avons:

$$\alpha_i(t)=\mathbb{P}(X_t=i,y_{1:t})$$

Ainsi: 

$$\alpha_i(1)=\mathbb{P}(X_1=i,y_{1:1})$$

$$=\mathbb{P}(X_1=i,y_1)=\mathbb{P}(y_1|X_1=i)\mathbb{P}(X_1=i)=b_i(y_1)\pi_i$$

donc : $\boxed{\alpha_i(1)=b_i(y_1)\pi_i}$

De plus: 

$$\alpha_i(t+1)=\mathbb{P}(X_{t+1}=i,y_{1:t+1})$$

Par les probas totales: 

$\mathbb{P}(X_{t+1}=i,y_{1:t+1})=\sum_{j={1:T}}[\mathbb{P}(X_t=j,X_{t+1}=i,y_{1:t},y_{t+1})]$

***(1)***: $\alpha_i(t+1)=\sum_{j={1:T}}[\mathbb{P}(X_t=j,y_{1:t})\mathbb{P}(X_{t+1}=i,y_{t+1}|X_t=j,y_{1:t})]$

On remarque alors que: 
- ***(2)***:  $\mathbb{P}(X_t=j,y_{1:t})=\alpha_j(t)$
- Grâce aux hypothèses de CMC on à 
  $$\mathbb{P}(X_t=j,X_{t+1}=i,y_{1:t},y_{t+1})=\mathbb{P}(X_t=j)\mathbb{P}(X_{t+1}=i|X_t=j)\mathbb{P}(y_{t}|X_t=j)\mathbb{P}(y_{t+1}|X_{t+1}=i)=\mathbb{P}(X_{t+1}=i,y_{t+1}|X_t=j,y_{1:t})]\mathbb{P}(X_t=j,y_{1:t})$$

  or:
	$$\mathbb{P}(X_t=j,y_{1:t})=\mathbb{P}(X_t=j)\mathbb{P}(y_t|X_t=j)$$

  d'où: 
  $$\mathbb{P}(X_{t+1}=i,y_{t+1}|X_t=j,y_{1:t})]=\mathbb{P}(X_{t+1}=i|X_t=j)\mathbb{P}(y_{t+1}|X_t=j)$$
- Avec la matrice de transition on a ***(3)***: 
  $$\mathbb{P}(X_{t+1}=i|X_t=j)=A'_{(i,j)}=A_{(j,i)}=a_{ji}$$
- Et la probabilité d'émission ***(4)***:
  $$\mathbb{P}(y_{t+1}|X_t=j)=b_j(y_{t+1})$$

En réunissant les formules ***3*** et ***4*** dans ***1*** avec ***2***, on a enfin: 
$$\boxed{\alpha_i(t+1)=\sum_{j={1:T}}[\alpha_j(t)a_{ji}b_j(y_{t+1})]=b_j(y_{t+1})\sum_{j={1:T}}[\alpha_j(t)a_{ji}]}$$

### Backward:
------------
On a:

$$\beta_i(t)=\mathbb{P}(y_{t+1:T}|X_t=i)$$

Pour l'initialisation: 

$$\boxed{\beta_i(T)=\mathbb{P}(y_{T+1:T}|X_T=i):=1}$$

Pour la récurence par les proba totales ***(5)***: 
$$\beta_i(t)=\sum_{j={1:T}}[\mathbb{P}(X_{t+1},y_{t+1:T}|X_t=i)$$

On remarque que:
- $$\mathbb{P}(X_t=i,X_{t+1},y_{t+1:T})=\mathbb{P}(X_{t+1},y_{t+1:T}|X_t=i)\mathbb{P}(X_t=i)$$
- $$\mathbb{P}(X_t=i,X_{t+1},y_{t+1:T})=\mathbb{P}(X_t=i)\mathbb{P}(X_{t+1}=j|X_t=i)\mathbb{P}(y_{t+1}|X_{t+1}=j)\mathbb{P}(y_{n+2:N}|x_{n+1})$$

on en déduit que ***(6)***: 
$$\mathbb{P}(X_{t+1},y_{t+1:T}|X_t=i)=\mathbb{P}(X_{t+1}=j|X_t=i)\mathbb{P}(y_{t+1}|X_{t+1}=j)\mathbb{P}(y_{n+2:N}|X_{n+1}=j)$$

On remarque également:
- $\mathbb{P}(X_{t+1}=j|X_t=i)=a_{ij}$ (cf. forward)
- $\mathbb{P}(y_{t+1}|X_{t+1}=j)=b_j(y_{t+1})$ (probabilité d'émission)
- $\mathbb{P}(y_{n+2:N}|X_{n+1}=j)=\beta_j(t+1)$

En injectant ces trois dernière remaques dans ***6*** puis ***5***:
$$\boxed{\mathbb{P}(X_{t+1},y_{t+1:T}|X_t=i)=\sum_{j={1:T}}[a_{ij}b_j(y_{t+1})\beta_j(t+1)]}$$

## Question 2:
--------------

### 1. Estimation des paramètres
La loi à priori à priori $\pi$ peut être aproximé dans un premier temps la fréquence, en tant qu'application du théorème de la loi des grand nombre avec T grand.
Ainsi: 
$$\pi_i=\mathbb{P}(X_t=i)$$
Est appoximé par:
$$\hat{\pi}_i=\frac{|\{t | X_t=i\}|}{T}$$

En observant le code dans la fonction \textit{HMC\_Learning\_parameters.py} nous remarquons que c'est ce qui est fait, on considère tou les subset du dataset et on compte le nombre d'occurance d'une catégorie de notre ensemble $\Omega_X$ avant de diviser par le nombre total de données en considerant toute les catégorie. Soit donc le pseudo code suivant:
```{}
- Pour tout subset du dataset
    - élement Zi du subset de catégorie x:
        Ajouter 1 à pi de la catégorie x
- Calculer sum_pi comme la somme de tous les pi (nombre d'élément dans tous les dataset) (cardial) 
- Pour tout x catégorie dans l'ensemble des catégories
    Diviser pi de la catégorie x par sum_pi
```
Qui revient bien à ce qui est fait dans le code python, dans ce code la manipulation de dictionaire implique tout de même la création dynamique des catégories ce qui correspond aux ```not in``` 

La procédure pour l'estimation de $A$ (matrice de transition) et $B$ (probabilité d'émission) suit le même principe
pour $A$:
$$A_{ij}=\mathbb{P}(X_t=j|X_{t-1}=i)=\frac{\mathbb{P}(X_t=j,X_{t-1}=i)}{\mathbb{P}(X_{t-1}=i)}$$
$$\hat{A_{ij}}=\frac{|\{t| X_t=j \cap X_{t-1}=i\, t\in \{2:T\}\}|}{|\{t| X_{t-1}=i, t\in \{2:T\}\}|}=\frac{nombre\ de\ transition\ de\ la\ catégorie\ i\ à\ j}{nombre\ total\ de\ transitions\ depuis\ l'état\ i}$$

pour $B$:
$$b_{i}(y)=\mathbb{P}(Y_t=y|X_{t}=i)=\frac{\mathbb{P}(Y_t=y,X_{t}=i)}{\mathbb{P}(X_{t}=i)}$$
$$\hat{b_i(y)}=\frac{|\{t| Y_t=y \cap X_{t}=i\, t\in \{1:T\}\}|}{|\{t| X_{t-1}=i, t\in \{1:T\}\}|}=\frac{occurance\ de\ l'observation\ y\ correspondant\ à\ un\ état\ i}{nombre\ d'observations\ correspondant\ à\ un\ état\ i}$$




### 2. Interet des dictionnaires
Un dictionnaire est un objet python où les données sont associés à des clés contrairement au liste ou les données sont dans un ordre. Le premier avantage est il me semble algorithmique car les dictionaire sont codées avec une table de hachage, le temps de recherche d'un élément est en complexité $O(1)$ contrairement au listes qui si elles ne sont pas triés sont en $O(n)$ ou au mieux en $O(log(n))$ aprés un tri.
Dans notre cas un autre avantage est la compréhension et la facilité d'acces dans cet objet. En effet on travail ici avec des mots, si nous utilisions des listes il faudrait définir un index faisant le lien entre un mot et un indice, indice que l'on utilise ensuite pour mettre les données au bon endroit dans la liste. Ce qui n'est pas optimal car la recherche dans la liste pour l'ajout d'un index sera en $O(n_1)$ et la sauvegarde en $O(n_2)$ alors que dans un dictionaire l'absence de présence est en $O(1)$ et l'ajout d'un élement au pire en $O(n_2). Ainsi l'acces d'une donnée à un mot grace à ce mot directement dans le dictionnaire est tout de même bien plus simple non seulement d'un points de vu algorithmique que d'un point de vu de compréhension.


### 3. Estimation par maximum de vraissemblance (EMV)
#### Pour $\pi_i$:

On a :
$$\pi_i=\mathbb{P}(X_t=i)$$
C'est la probabilité qu'un élément de $X=(X_1, ... X_T$ soit dans la catégorie $i$. On peut donc imaginer qu'un élement $X_t$ de $X$ suit une loi binomiale de paramètres $\pi_i$ d'être dans la catégorie $i$. 
On a alors: 
$$X_t\sim B(\pi_i)$$
On en déduit la vraissemblance:
$$L_T(X_{1:T},\pi_i)=\prod_{t=1}^{T}{ \mathbb{P}_{\pi_i}(X_t=i)}=\pi_i^s(1-\pi_i)^{T-s}$$
Où $s$ représente ici le nombre de "succes" c'est à dire de $X_t$ de classe $i$, Ainsi l'estimateur du maximum de vraissemblances $\hat{\pi_i}$ est trouvé par:
$$\frac{\delta L_T}{\delta \hat{\pi_i}}=\hat{\pi_i}^{s-1}(1-\hat{\pi_i})^{T-s-1}(s-T\hat{\pi_i})=0 \iff \hat{\pi_i}=\frac{s}{T}$$
Car (en passant par le log pour me simplifier les calculs):
$$\frac{\delta^2 L_T}{\delta^2 \hat{\pi_i}}=-s\frac{1}{\hat{pi_i}}-(T-s)\frac{1}{1-\hat{pi_i}}=-2T<0$$
Conclusion l'estimateur du maximum de vraissemblance:
$$\boxed{\hat{\pi_i}=\frac{s}{T}}$$
C'est bien l'estimateur utilisé dans le code d'après ce que nous avons décrit au $2.1$


#### Pour $A_{ij}$:
On a: 
$$A_{ij}=\mathbb{P}(X_t=j|X_{t-1}=i)$$
Il s'agit en fait de la probabilité de transition de l'état i à j en une étape. Comme précedement on peut modéliser la situation comme $Z_k(i)$ la variable aléatoire valant 1 si $X_t=j$ et $0$ sinon dans $X_{t-1}=i$, loi de $X_t=j$ conditionnellement à $X_{t-1}=i$. Ici $k$ à valeur entre 0 et le nombre de variable de $X$ de classe $i$ que nous notons $n_i$.
On a: 
$$Z_k(i)\sim B(a_{ij})$$
En reprennant le calcul précédant on a:
- $s$ que nous notons maintenant $n_{i\rightarrow j}$ le nombre de varaible $X_t$ de catégorie $j$ tel que $X_{t-1}=i$
- $T$ que nous notons maintenant $n_i$ le nombre de $X_t$ valant $i$ 

Ainsi le résonnement précédant s'applique aussi ici et: 
$$\boxed{\hat{a_{ij}}=\frac{n_{i\rightarrow j}}{n_i}}$$
C'est bien l'estimateur utilisé dans le code d'après ce que nous avons décrit au $2.1$


#### Pour $b_i(y)$:
On a: 
$$b_i(y)=\mathbb{P}(Y_t=y|X_t=i)$$
Il s'agit de la probabilité qu'un observation prenne la valeur $y$ sachant que la catégorie qui lui est associé vaut $i$. Ici $y$ est un mot de l'espace $\Omega_y$ fini (l'ensemble des mots d'une langue est un espace fini). En notant $W_m(i)$  la variable aléatoire valant 1 si $Y_t=y$ et $0$ sinon dans $X_{t}=i$, loi de $Y_t=y$ conditionnellement à $X_{t}=i$. Ici $m$ à valeur entre $0$ et le nombre de mot de catégorie $i$ noté $n_i$.
On a: 
$$W_m(i)\sim B(b_{i}(y))$$
En reprennant le calcul précédant on a:
- $s$ que nous notons maintenant $n_{y}$ le nombre de varaible $Y_t$ valant le mot $y$ de catégorie $X_{t}=i$
- $T$ que nous notons maintenant $n_i$ le nombre de $X_t$ valant $i$ 

Ainsi le résonnement précédant s'applique aussi ici et: 
$$\boxed{\hat{b_{i}(y)}=\frac{n_{y}}{n_i}}$$
C'est bien l'estimateur utilisé dans le code d'après ce que nous avons décrit au $2.1$
 

## Question 3:
--------------


## Question 4:
--------------

CHUNK 2000
HMC Accuracy on CoNLL 2000 Chunking: 7.191253139709147 %
KW: 6.677254679523543
UW: 14.052089642640823
Training time:  0.16566085815429688

CHUNK 2003
HMC Accuracy on CoNLL 2003 POS Tagging: 5.62357201362245 %
KW: 5.207347552837831
UW: 8.933050356936135
Training time:  0.3436698913574219

NER 2000


NER 2003

Results:
Precision: 0.624182603331277
Recall: 0.8646385233293454
F1: 0.7249928346231013 

Results KW:
Precision: 0.8724066390041494
Recall: 0.8661174047373842
F1: 0.8692506459948321 

Results UW:
Precision: 0.005601034037052995
Recall: 0.52
F1: 0.011082693947144074


POS 2000
HMC Accuracy on CoNLL 2000 POS Tagging: 5.359140511218513 %
KW: 1.9444129325014217
UW: 50.938824954572986
Training time: 0.18027377128601074


POS 2003
HMC Accuracy on CoNLL 2000 POS Tagging: 9.912919774108715 %
KW: 3.979520031059664
UW: 57.09048813428516


POS UD ENGLISH
HMC Accuracy on CoNLL 2000 POS Tagging: 11.161141217723937 %
KW: 6.222550767491413
UW: 65.26917579799905
Training time: 0.27544498443603516


| type de tag | POS | Chunking | NER |
| :---------- |:----:|:-------:|:---:|
| type de tag | POS | Chunking | NER |

## Question 5:
--------------

## Question 6:
--------------

## Question 7:
--------------

## Question 8:
--------------